In [ ]:
from pathlib import Path
import os
from joblib import Memory
from sklearn.datasets import load_svmlight_file
from scipy.sparse import csc_matrix
from scipy.special import expit
from scipy import sparse
from sklearn.preprocessing import normalize
from copy import copy
from sklearn import svm
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn import tree
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from scipy.stats import pearsonr

import pyarrow.feather as feather
import matplotlib.pyplot as plt
import plotnine as p9
import pandas as pd
import numpy as np

rand_state = 5991

In [ ]:
# FUNCTIONS
def split_data(xx, yy, testsize = 1000):
    xtrain, xtest, y_train, y_test = train_test_split(xx,
                                                      yy,
                                                      test_size = testsize,
                                                      random_state = rand_state)
    xtrain, xvalid, y_train, y_valid = train_test_split(xtrain, y_train, 
                                                        test_size = testsize,
                                                        random_state = rand_state)
    y_train = np.array(y_train).reshape(-1, 1)
    y_test = np.array(y_test).reshape(-1, 1)
    y_valid = np.array(y_valid).reshape(-1, 1)

    print(" SHAPE of xtrain:", xtrain.shape)
    print("SHAPE of y_train:", y_train.shape)
    print("  SHAPE of xtest:", xtest.shape)
    print(" SHAPE of y_test:", y_test.shape)
    print(" SHAPE of xvalid:", xvalid.shape)
    print("SHAPE of y_valid:", y_valid.shape)

    return xtrain, y_train, xtest, y_test, xvalid, y_valid 

def get_acc_auc(y, p):
    acc = np.sum(y == p) / len(y)
    auc = roc_auc_score(y, p)
    return acc, auc

In [ ]:
# Paths
idcsc = "postprocess/idcsc.feather"
cipcsc = "postprocess/cipcsc.feather"
geoloccsc = "postprocess/geoloccsc.feather"
instdemocsc = "postprocess/instdemocsc.feather"
studdemocsc = "postprocess/studdemocsc.feather"
pcipcsc = "postprocess/pcipcsc.feather"
numcsc = "postprocess/numcsc.feather"
tarcsc = "postprocess/tarcsc.feather"
fulldf = "postprocess/fulldf.feather"

num_only = pd.read_csv("C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/numeric_only.csv")


In [ ]:
id_csc = feather.read_feather(idcsc)
cip_csc = feather.read_feather(cipcsc)
geolocation_csc = feather.read_feather(geoloccsc)
inst_demographic_csc = feather.read_feather(instdemocsc)
stud_demographic_csc = feather.read_feather(studdemocsc)
pcip_csc = feather.read_feather(pcipcsc)
num_csc = feather.read_feather(numcsc)
tar_csc = feather.read_feather(tarcsc)

full_df = feather.read_feather(fulldf)

missing = pd.DataFrame(full_df.isna().sum())
missing.reset_index(inplace=True)
missing[missing[0] > 0]

In [ ]:
# Do the ids column in the target frame match the the ids column in the full frame for each unique UNITID?
df = pd.merge(tar_csc[["ids", "UNITID"]],
         full_df[["ids", "UNITID", "INSTNM"]],
         how = "left", left_on = "UNITID", right_on = "UNITID",
         suffixes=("_target", "_full"))
df["diff"] = df["ids_target"] - df["ids_full"]
df.loc[df["diff"] > 0 ]
# Yes

In [ ]:
# Changing Majority Pell to have the value of 1 and Minority Pell to have the value of 0.
# check before
df = pd.merge(id_csc, tar_csc, how = "left",
              left_on = "UNITID", right_on = "UNITID",
              suffixes = ["_id", "_target"])
df[["PELLCAT_id", "PCTPELL", "PELLCAT_target"]]

pc_dict = { 1: "MINPELL", 0: "MAJPELL" } # originally Minority Pell equals 1, and Majority Pell equals 0.
pc_dict2 = { "MINPELL": 0, "MAJPELL": 1 } # I want to change Minority Pell to equal 0 and Majority Pell to equal 1.
tar_csc["PELLCAT"] = tar_csc["PELLCAT"].map(pc_dict)
tar_csc["PELLCAT"] = tar_csc["PELLCAT"].map(pc_dict2)
id_csc["PELLCAT"] = id_csc["PELLCAT"].map(pc_dict)
id_csc["PELLCAT"] = id_csc["PELLCAT"].map(pc_dict2)

# check after
df = pd.merge(id_csc, tar_csc, how = "left",
              left_on = "UNITID", right_on = "UNITID",
              suffixes = ["_id", "_target"])
df[["PELLCAT_id", "PCTPELL", "PELLCAT_target"]]

In [ ]:
full_df.head(3)


In [ ]:
ys = tar_csc["PELLCAT"].copy()
xs = full_df.drop(["ids", "UNITID"], axis = 1).copy()
print("SHAPE of xs:", xs.shape)
print("SHAPE of ys:", ys.shape)

In [ ]:
xtrain, y_train, xtest, y_test, xvalid, y_valid = split_data(xs, ys, testsize = 1000)

### Checking for non-linear associations between individual features and the binary target variable

In [ ]:
print(xtrain.shape)

xnot = list(xtrain.columns[xtrain.columns.str.endswith("ASSOC") | xtrain.columns.str.endswith("BACHL") |
                      xtrain.columns.str.endswith("CERT1") | xtrain.columns.str.endswith("CERT2") |
                      xtrain.columns.str.endswith("CERT4")])
xnot2 = ["INSTNM", "ACCREDAGENCY", "ACCREDCODE", "CITY", "Season", "STABBR", "T4APPROVALDATE", "ZIP"]
xnot.extend(xnot2)

print("variables to be removed:")
print(len(xnot))
print("remaining variables:")

features = xtrain.columns.drop(xnot)

print(len(features))

missing = pd.DataFrame(xtrain[features].isna().sum())
missing.reset_index(inplace=True)
missing[missing[0] > 0]

In [ ]:
scaler = StandardScaler()
poly = PolynomialFeatures(3)

scaler.fit(xtrain[features])
xtrain = pd.DataFrame(scaler.transform(xtrain[features]), columns = features)
xvalid = pd.DataFrame(scaler.transform(xvalid[features]), columns = features)
xtest = pd.DataFrame(scaler.transform(xtest[features]), columns = features)

log_reg = LogisticRegression(solver = 'saga',
                             random_state = rand_state,
                             penalty = 'l1',
                             class_weight = 'balanced',
                             max_iter = 1000)

print("xtrain before: ", xtrain.shape)

In [ ]:
# CREDIT: Dr. Vanderheyden wrote this code.
accuracies = []
for f in features:

    x = xtrain[f].values.reshape(-1, 1)
    y = y_train.reshape(-1, 1)
    ## LIN ##############################
    log_reg.fit(x, y)
    acc, auc = get_acc_auc(y, log_reg.predict(x))
    ## LOG #############################   
    xl = np.log(x - np.min(x) + 1)
    log_reg.fit(xl, y)
    lcc, luc = get_acc_auc(y, log_reg.predict(xl))
    if lcc / acc >= 1.1 or luc / auc >= 1.05:  # if bin accuracy is 110% of linear accuracy or ... AUC is 105% ...
        xtrain[f + ' log'] = xl
        xvalid[f + ' log'] = np.log(xvalid[f].values.reshape(-1, 1) - np.min(xtrain[f])+1)
        xtest[f + ' log'] = np.log(xtest[f].values.reshape(-1, 1) - np.min(xtrain[f])+1)
    ## EXP #############################   
    xe = np.exp(x)
    log_reg.fit(xe, y)
    ecc, euc = get_acc_auc(y, log_reg.predict(xe))
    if ecc / acc >= 1.1 or euc / auc >= 1.05: 
        xtrain[f + ' exp'] = xe
        xvalid[f + ' exp'] = np.exp(xvalid[f].values.reshape(-1, 1))
        xtest[f + ' exp'] = np.exp(xtest[f].values.reshape(-1, 1))
    ## POLY ############################# 
    poly.fit(x)
    xp = poly.transform(x)
    log_reg.fit(xp, y)
    pcc, puc = get_acc_auc(y, log_reg.predict(xp))
    if pcc / acc >= 1.1 or puc / auc >= 1.05:  # if bin accuracy is 110% of linear accuracy or ... AUC is 105% ...
        xtrain[f + ' p2'] = x**2
        xtrain[f + ' p3'] = x**3
        xvalid[f + ' p2'] = (xvalid[f].values)**2
        xvalid[f + ' p3'] = (xvalid[f].values)**3
        xtest[f + ' p2'] = (xtest[f].values)**2
        xtest[f + ' p3'] = (xtest[f].values)**3
    ## BIN #############################
    xmin = x.min()
    rnge = x.max() - xmin
    xtrn = 0 + ((x - xmin) > 1 * rnge / 10) + ((x - xmin) > 2 * rnge / 10) + ((x - xmin) > 3 * rnge / 10) + ( # the objects in each
                (x - xmin) > 4 * rnge / 10) + ((x - xmin) > 5 * rnge / 10) + ((x - xmin) > 6 * rnge / 10) + ( # bracket returns true
                (x - xmin) > 7 * rnge / 10) + ((x - xmin) > 8 * rnge / 10) + ((x - xmin) > 9 * rnge / 10)     # or false 
    xval = 0 + ((xvalid[f] - xmin) > 1 * rnge / 10) + ((xvalid[f] - xmin) > 2 * rnge / 10) + ((xvalid[f] - xmin) > 3 * rnge / 10) + (
                (xvalid[f] - xmin) > 4 * rnge / 10) + ((xvalid[f] - xmin) > 5 * rnge / 10) + ((xvalid[f] - xmin) > 6 * rnge / 10) + (
                (xvalid[f] - xmin) > 7 * rnge / 10) + ((xvalid[f] - xmin) > 8 * rnge / 10) + ((xvalid[f] - xmin) > 9 * rnge / 10)
    xtst = 0 + ((xtest[f] - xmin) > 1 * rnge / 10) + ((xtest[f] - xmin) > 2 * rnge / 10) + ((xtest[f] - xmin) > 5 * rnge / 10) + (
                (xtest[f] - xmin) > 3 * rnge / 10) + ((xtest[f] - xmin) > 4 * rnge / 10) + ((xtest[f] - xmin) > 6 * rnge / 10) + (
                (xtest[f] - xmin) > 7 * rnge / 10) + ((xtest[f] - xmin) > 8 * rnge / 10) + ((xtest[f] - xmin) > 9 * rnge / 10)
    encoder = TargetEncoder()
    encoder.fit(xtrn, y)
    xb = encoder.transform(xtrn)
    log_reg.fit(xb, y)
    bcc, buc = get_acc_auc(y, log_reg.predict(xb))
    if bcc / acc >= 1.1 or buc / auc >= 1.05: # if bin accuracy is 110% of linear accuracy or ... AUC is 105% ...
        xtrain[f + ' Bin'] = xb
        xvalid[f + ' Bin'] = encoder.transform(xval)
        xtest[f + ' Bin'] = encoder.transform(xtst)
    ## COMPLETION #############################
    lDa = lcc / acc
    eDa = ecc / acc
    pDa = pcc / acc
    bDa = bcc / acc
    lda = luc / auc
    eda = euc / auc
    pda = puc / auc
    bda = buc / auc
    accuracies.append([f, acc, lcc, ecc, pcc, bcc, auc, luc, euc, puc, buc, lDa, eDa, pDa, bDa, lda, eda, pda, bda])
###############################################

colums = ['Feature','ACC: Linear', 'ACC: Log', 'ACC: Exp', 'ACC: Poly3','ACC: Bin',
                    'AUC: Simple Linear', 'AUC: Log', 'AUC: Exp','AUC: Poly3', 'AUC: Bin',
                    "ACC: LOG / Linear", "ACC: EXP / Linear", "ACC: Poly3 / Linear", "ACC: Bin / Linear",
                    "AUC: LOG / Linear", "AUC: EXP / Linear", "AUC: Poly3 / Linear", "AUC: Bin / Linear"]
accDf = pd.DataFrame(accuracies, columns = colums)
accDf

In [ ]:
print(xtrain.shape)
accDf.head(5)
accDf.loc[:, ["ACC: LOG / Linear", "ACC: EXP / Linear", "ACC: Poly3 / Linear", "ACC: Bin / Linear",
           "AUC: LOG / Linear", "AUC: EXP / Linear", "AUC: Poly3 / Linear", "AUC: Bin / Linear"]].sort_values("ACC: LOG / Linear", ascending = False).head(5)
# accDf.sort_values("AUC: Log").head(20)

### Checking the Correlation of features

In [ ]:
from scipy.stats import pearsonr

corrs = []
contFeat = list(xtrain.columns)
contFeat_length = len(contFeat)


for i in range(contFeat_length):
    for j in range(i + 1, contFeat_length):
        feati = xtrain[contFeat[i]].values.flatten()
        featj = xtrain[contFeat[j]].values.flatten()

        corr, _ = pearsonr(feati, featj)
        corrs.append([feati, featj, corr, contFeat[i], contFeat[j]])


In [ ]:
correl = pd.DataFrame(corrs, columns = ['f1', 'f2', 'P Corr', "feat1", "feat2"])
################################
num_var_g50 = len(correl[abs(correl['P Corr']) > 0.5])
pct_var_g50 = num_var_g50 / (contFeat_length * (contFeat_length - 1) / 2)
print(xtrain.shape)
print('Total number of features pairs: )', num_var_g50)
print('Number of features pairs with absolute Pearson Correl above 0.5: )', num_var_g50)
print('Percent of features pairs with absolute Pearson Correl above 0.5: )', pct_var_g50)

In [ ]:
pct_var_g50 * 100
correl[abs(correl['P Corr']) > 0.5]

# Modelling

#### Logistic Regression

In [ ]:
from sklearn.metrics import roc_auc_score

log = LogisticRegression(solver = 'saga',
                         random_state = rand_state, 
                         penalty = 'l1', 
                         C = 0.05, 
                         class_weight = 'balanced',
                         max_iter = 3000)
log.fit(xtrain, y_train)

Tprob = log.predict_proba(xtrain)
vprob = log.predict_proba(xvalid)
tprob = log.predict_proba(xtest)

print('Train AUC and Accuracy: ', roc_auc_score(y_train, Tprob[:,1]), 1 - np.mean(abs(y_train - log.predict(xtrain))))
print('Train AUC and Accuracy: ', roc_auc_score(y_test, tprob[:,1]), 1 - np.mean(abs(y_test - log.predict(xvalid))))
print('Val AUC and Accuracy: ', roc_auc_score(y_valid, vprob[:,1]), 1 - np.mean(abs(y_valid - log.predict(xtest))))
print('Number of Weights equal to zero: ', np.sum(log.coef_==0))

In [ ]:

from sklearn.ensemble import RandomForestClassifier

leafs = [2, 5, 10, 20, 30, 40]
depths = [5, 6, 7, 8, 9, 10, 11, 12]

for l in leafs:
    for d in depths:
        forest = RandomForestClassifier(n_estimators=1000, criterion='entropy', max_depth=d, min_samples_leaf=l, class_weight='balanced_subsample', random_state = 67)

        forest.fit(xtrain, y_train)

        Tpred = forest.predict(xtrain)
        vpred = forest.predict(xval)

        Tprob = forest.predict_proba(xtrain)
        vprob = forest.predict_proba(xval)
        tprob = forest.predict_proba(xtest)

        print('Train AUC and Accuracy: ', l, d, roc_auc_score(y_train, Tprob[:,1]), 1 - np.mean(abs(y_train - Tpred)))
        print('Val AUC and Accuracy: ', l, d, roc_auc_score(y_val, vprob[:,1]), 1 - np.mean(abs(y_val- vpred)))
        print()  